In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_table('training_dataset.txt', sep = '	')
classify = pd.read_table('test_dataset.txt')

C:\Users\irina\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
C:\Users\irina\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [3]:
df.head()

,name,result
0,1.вода аква минерале газ 0.,НЕ Сигареты ROTHMANS
1,"сигареты""keht""",НЕ Сигареты ROTHMANS
2,сигареты стиль rose ssl jade (жадэ розовый),НЕ Сигареты ROTHMANS
3,мин вода нарзан 0.5л пэт,НЕ Сигареты ROTHMANS
4,пирожок печен с мясом,НЕ Сигареты ROTHMANS


In [4]:
# Для краткости дадим классам более короткие наименования
dict_classes = {'НЕ Сигареты ROTHMANS': 'c1',
               'Сигареты ROTHMANS DEMI CLICK (кнопка / капсула / зеленый / лайм / ментол)':'c2',
               'Сигареты ROTHMANS DEMI CLICK (кнопка / капсула) Plus Blue (amber, мандарин, оранж, апельсин)':'c3',
               'Сигареты ROTHMANS DEMI':'c4',
               'Сигареты ROTHMANS DEMI SILVER (серый)':'c5',
               'Сигареты ROYALS BY ROTHMANS':'c6',
               'Сигареты ROTHMANS DEMI CLICK (кнопка / капсула) Aero Blue (фиолетовый / сиреневый / ягоды)':'c7',
               'Сигареты ROTHMANS (неизвестно или нет в списке)':'c8'}

df['result'] = df['result'].map(dict_classes)

In [5]:
# Количество примеров для каждой категории
df['result'].value_counts()

c1    9200
c2     222
c3     185
c4     172
c5      76
c6      75
c7      59
c8       9
Name: result, dtype: int64

##### Поскольку перед нами стоит задача определения категории товара по тексту из чека, попробуем воспользоватся некоторыми алгоритмами nlp. Будем использовать меру TF-IDF, предварительно переведя текст в векторный формат

In [6]:
# Для начала необходимо очистить текст от ненужных символов в наименовании товарной позиции

def clean_text(text):

    REPLACE_BY_SPACE= re.compile('[/(){}""\[\]\|@;]')
    BAD_SYMBOLS= re.compile('[#+_]')
    
    text = text.lower() 
    text = REPLACE_BY_SPACE.sub(' ', text)
    text = BAD_SYMBOLS.sub('', text) 
    return text

In [7]:
# Применим функцию к обучающей выборке
df['name'] = df['name'].apply(clean_text)

In [8]:
# разобьем выборку на train и test
train = df.sample(frac = 0.7, random_state = 200)
test = df.drop(train.index)

In [9]:
# Разбивка на train и test с выбранным sub_sample нам подходит - как в train, так и в test есть примеры обоих классов
print(train['result'].value_counts())
print(test['result'].value_counts())

c1    6454
c2     145
c3     142
c4     124
c5      46
c6      46
c7      37
c8       5
Name: result, dtype: int64
c1    2746
c2      77
c4      48
c3      43
c5      30
c6      29
c7      22
c8       4
Name: result, dtype: int64


In [10]:
# Выделим target и определим список классов
y_train = train['result']
y_test = test['result']

X_train = train['name']
X_test = test['name']

classes = list(df['result'].value_counts().index)

##### Попробуем обучить несколько моделей на обучающей выборке. Качество будем определять по выборке, отложенной ранее

In [11]:
from sklearn.pipeline import Pipeline
import collections
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier #SVM (Support Vector Machine)

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=0.001, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

accuracy 0.9816605535178393
              precision    recall  f1-score   support

          c1       0.99      1.00      1.00      2746
          c2       0.83      0.87      0.85        77
          c3       0.97      0.79      0.87        43
          c4       0.73      0.92      0.81        48
          c5       1.00      0.73      0.85        30
          c6       0.91      0.72      0.81        29
          c7       1.00      0.50      0.67        22
          c8       0.00      0.00      0.00         4

    accuracy                           0.98      2999
   macro avg       0.80      0.69      0.73      2999
weighted avg       0.98      0.98      0.98      2999



C:\Users\irina\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)


y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

C:\Users\irina\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\irina\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.9859953317772591
              precision    recall  f1-score   support

          c1       1.00      1.00      1.00      2746
          c2       0.90      0.95      0.92        77
          c3       0.95      0.88      0.92        43
          c4       0.67      0.92      0.77        48
          c5       0.97      0.93      0.95        30
          c6       0.90      0.62      0.73        29
          c7       0.94      0.73      0.82        22
          c8       0.00      0.00      0.00         4

    accuracy                           0.99      2999
   macro avg       0.79      0.75      0.76      2999
weighted avg       0.99      0.99      0.99      2999



In [13]:
import xgboost as XGB

Boost = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', XGB.XGBClassifier(n_jobs=-1, reg_alpha = 0.1, reg_lambda = 0.1,
                          min_samples_leaf = 2, min_samples_split = 5, learning_rate = 0.01, n_estimators = 700,
                          subsample = 0.9)),
               ])
Boost.fit(X_train, y_train)


y_pred = Boost.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=classes))

accuracy 0.9816605535178393
              precision    recall  f1-score   support

          c1       1.00      1.00      1.00      2746
          c2       0.80      0.84      0.82        77
          c3       0.92      0.77      0.84        43
          c4       0.63      0.88      0.73        48
          c5       1.00      0.83      0.91        30
          c6       0.96      0.76      0.85        29
          c7       0.79      0.50      0.61        22
          c8       0.50      0.25      0.33         4

    accuracy                           0.98      2999
   macro avg       0.82      0.73      0.76      2999
weighted avg       0.98      0.98      0.98      2999



In [14]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, y_pred, labels = classes)

array([[[ 247,    6],
        [   1, 2745]],

       [[2906,   16],
        [  12,   65]],

       [[2953,    3],
        [  10,   33]],

       [[2926,   25],
        [   6,   42]],

       [[2969,    0],
        [   5,   25]],

       [[2969,    1],
        [   7,   22]],

       [[2974,    3],
        [  11,   11]],

       [[2994,    1],
        [   3,    1]]], dtype=int64)

##### Из построенной матрицы ошибок можно увидеть, что модель часто совершает ложные срабатывания в С2 и С4, а также довольно часто не распознает С7 и С3 (в чуть меньшей степени)

In [15]:
confusion_matrix(y_test, y_pred, labels = classes)

array([[2745,    0,    0,    1,    0,    0,    0,    0],
       [   2,   65,    1,    9,    0,    0,    0,    0],
       [   2,    4,   33,    3,    0,    0,    1,    0],
       [   2,    3,    0,   42,    0,    0,    0,    1],
       [   0,    0,    1,    4,   25,    0,    0,    0],
       [   0,    2,    0,    3,    0,   22,    2,    0],
       [   0,    6,    1,    4,    0,    0,   11,    0],
       [   0,    1,    0,    1,    0,    1,    0,    1]], dtype=int64)

##### Из построенной расширенной матрицы ошибок для множественной классификации можно увидеть, что модель часто "путает" С2, C5 и С7 c C4

##### В качестве дополнительной метрики можно использовать ROC_AUC:

In [16]:
from sklearn.metrics import roc_auc_score
Roc = pd.DataFrame(data = {'test':y_test, 'pred':y_pred})
macro_avg_auc = []
for i in classes:
    Roc = pd.DataFrame(data = {'test':y_test, 'pred':y_pred})
    Roc.loc[Roc['test'] != i, 'test'] = 0
    Roc.loc[Roc['test'] == i, 'test'] = 1
    
    Roc.loc[Roc['pred'] != i, 'pred'] = 0
    Roc.loc[Roc['pred'] == i, 'pred'] = 1
    auc_i = round(roc_auc_score(Roc['test'], Roc['pred']), 3)
    print(i, 'AUC_ROC:', auc_i)
    macro_avg_auc.append(auc_i)
print('MACRO_AUC_ROC:', round(np.array(macro_avg_auc).mean(), 3))

c1 AUC_ROC: 0.988
c2 AUC_ROC: 0.919
c3 AUC_ROC: 0.883
c4 AUC_ROC: 0.933
c5 AUC_ROC: 0.917
c6 AUC_ROC: 0.879
c7 AUC_ROC: 0.749
c8 AUC_ROC: 0.625
MACRO_AUC_ROC: 0.862


##### Попробуем найти наиболее важные слова, которые характеризуют каждый класс:

In [17]:
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)

dict_fit = vectorizer.get_feature_names() # bag_of_words

In [18]:
tfidf = TfidfTransformer()
X_train_vect_tfidf = tfidf.fit_transform(X_train_vect)

In [19]:
# Функция, которая принимает на вход метку класса и возвращает 10 слов с наивысшей мерой tf-idf
def imp_words(C):
    y_train_res_ind = y_train.reset_index()
    X_class_index = y_train_res_ind[y_train_res_ind['result'] == C].index
    
    list_words_j = []
    for i in X_class_index:
        for j in X_train_vect_tfidf.toarray()[i].argsort()[-10:][::-1]:
            list_words_j.append(dict_fit[j])
    
    c = collections.Counter()
    for w in list_words_j:
        c[w] += 1
    return c

In [20]:
# most_imp_w_1 = imp_words('c1')  -  для данного класса довольно долгие вычисления. Результаты приводятся без них
most_imp_w_2 = imp_words('c2')
most_imp_w_3 = imp_words('c3')
most_imp_w_4 = imp_words('c4')
most_imp_w_5 = imp_words('c5')
most_imp_w_6 = imp_words('c6')
most_imp_w_7 = imp_words('c7')
most_imp_w_8 = imp_words('c8')

In [21]:
for i,j in zip([most_imp_w_2, most_imp_w_3, most_imp_w_4,
          most_imp_w_5, most_imp_w_6, most_imp_w_7, most_imp_w_8], ['c2','c3','c4','c5','c6','c7','c8']):
    print(j)
    print(i.most_common(10))

c2
[('mab', 111), ('m520', 102), ('mac', 95), ('деми', 87), ('ротманс', 86), ('клик', 76), ('m41', 70), ('сигареты', 68), ('rothmans', 55), ('demi', 55)]
c3
[('m520', 107), ('m41', 92), ('mab', 81), ('деми', 81), ('ротманс', 79), ('сигареты', 73), ('амбер', 70), ('m3', 65), ('demi', 60), ('rothmans', 56)]
c4
[('m520', 114), ('mab', 111), ('m41', 104), ('mac', 78), ('m3', 74), ('деми', 62), ('ротманс', 60), ('rothmans', 59), ('demi', 59), ('сигареты', 54)]
c5
[('mab', 38), ('m520', 36), ('сигареты', 31), ('mac', 29), ('rothmans', 26), ('demi', 26), ('silver', 20), ('сильвер', 19), ('ротманс', 19), ('деми', 19)]
c6
[('m41', 37), ('m3', 34), ('ротманс', 33), ('m520', 33), ('деми', 32), ('m2', 24), ('mab', 23), ('сигареты', 19), ('роялс', 18), ('ёш', 17)]
c7
[('m520', 25), ('деми', 21), ('m41', 21), ('mab', 21), ('ротманс', 19), ('m3', 17), ('аэро', 16), ('rothmans', 14), ('demi', 14), ('сигареты', 13)]
c8
[('ротманс', 4), ('деми', 4), ('mab', 3), ('m520', 3), ('блю', 3), ('m41', 3), ('m3'

##### Видно, что классы c2 и с3 довольно сильно похожи межу собой по наиболее "важным" словам. Класс c7 не очень похож на эти 2. Классы c4 и с5 также имеют похожий состав "важных" слов

In [ ]:
# Разметим клиентов из выборки classify

In [24]:
classify['name'] = classify['name'].map(clean_text)

In [26]:
Boost_clf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', XGB.XGBClassifier(n_jobs=-1, reg_alpha = 0.1, reg_lambda = 0.1,
                          min_samples_leaf = 2, min_samples_split = 5, learning_rate = 0.01, n_estimators = 700,
                          subsample = 0.9)),
               ])
Boost_clf.fit(df['name'], df['result'])

y_pred_classify = Boost.predict(classify['name'])

In [29]:
y_clf = pd.DataFrame(data = {'result':y_pred_classify})

In [30]:
y_clf['result'].value_counts() # В целом, распределение классов довольно сильно похоже на обучающую выборку. 
# Однако доля класса c1 еще выше

c1    19796
c4       65
c2       64
c3       41
c5       18
c7        9
c6        5
c8        2
Name: result, dtype: int64

In [35]:
clf_result = pd.concat([classify['name'], y_clf['result']], axis = 1)

In [36]:
clf_result

,name,result
0,2000000964829 трихопол таб 250 мг n2,c1
1,сигареты парламент карат синий,c1
2,перец горький 1 шт,c1
3,сыр коса сочинский 120 гр 40,c1
4,мертенил таб 5мг є30,c1
5,"нектар донди 0,2л ¤блоко-виноград",c1
6,лейз грибы и сметана 80гр,c1
7,конь¤к российский трехлетний крым таврический ...,c1
8,6094 сигареты ¤ва белое золото кл,c1
9,минеральна¤ вода боржоми 0.5,c1


In [42]:
inv_dict_classes = {v: k for k, v in dict_classes.items()}
clf_result['result'] = clf_result['result'].map(inv_dict_classes)

In [44]:
clf_result.to_csv('predictions.txt', sep = '	')

##### Для решения данной задачи использовались различные алгоритмы классификации (градиентный бустинг, логистическая регрессия, SVM) на основе рассчитанной меры TF-IDF. Помимо того, что мера TF-IDF довольно просто рассчитывается, она еще хороша тем, что позволяет определять слова характерные только для какого-то конкретного класса.

##### В качестве метрик классификации использовались : F-Мера и AUC_ROC. При расчете использовалось простое усреднение (macro) и веса классов не учитывались. В результате, каждый класс вносил равный вклад в итоговую метрику, что является более объективным в нашем случае.  Полученные значения - F-Мера = 0.76, AUC_ROC = 0.86 (XGBOOST)

##### В качестве улучшения построенного алгоритма классификации можно сделать следующее:
#####  -  Тщательнее подобрать гиперпараметры алгоритмов на кросс-валидации;
#####  -  Более качественно очистить выборку от различных ненужных символов, добавить исключение по стоп-словам (для tf-idf);
#####  -  Обогатить выборку данными (можно проверить значения метрик на меньших объемах выборки, постепенно увеличивая его);
#####  -  В качестве дополнения к бизнес-процессу - "спорные" ситуации, например когда модель выдала по нескольким классам примерно равные вероятности отправлять на ручную проверку.